# MINN-reservoir experiments

This notebook can be used to reproduced the results regarding the MINN-reservoir presented in the paper. 

The output will be the extra upper bounds to enanche the mechanistic model predicitons.

In [ ]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
import pandas as pd
from src.nn_model.amn_qp import *
from src.utils.hpo import hpo
from src.utils.import_data import *
from src.utils.import_GEM import *
from src.utils.training import *
from src.utils.plots import *
from src.utils.utils import *
from hydra import initialize, compose
from omegaconf import OmegaConf
import logging

# log hydra configuration file 

In [ ]:
# hydra in jupyter notebook
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

config_dir = "conf" # Adjust path
initialize(config_path=config_dir, job_name="notebook_logging", version_base="1.1")

# HERE YOU CAN DECIDE WHICH CONFIGURATION FILE TO LOG
# 

cfg = compose(config_name="MINN_reservoir") 

# Step 3: Log the configuration
logger.info("Logging configuration:")
logger.info(OmegaConf.to_yaml(cfg))


# evaluation pipeline: loocv + kfold

In [ ]:
# No clearML when using jupyter
task= None
# reproduciblility
seed = cfg.seed
fix_random_seed(seed=seed)

# load gem and data
n_distribution= cfg.gem.n_total_reactions
X, y, Vin, fit_model, reference = load_ishii(seed=seed, 
                                                dataset=cfg.dataset.dataset_name,
                                                fluxes_removed_from_reference=cfg.dataset.fluxes_removed_from_reference,
                                                fluxes_to_add_input = cfg.dataset.fluxes_to_add_input,
                                                kos_genes= cfg.dataset.kos_genes,
                                                gem = cfg.gem.gem_name)

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()

# Lists to store results for each LOO iteration
Q2_loo = []
MAE_loo = []
RMSE_loo = []
NE_loo = []

# List to store single left out predictions
Vref_true_loo = []
Vref_pred_loo = []
Vpred_final = []
Vin_reservoir_final = []
list_SV_loss_per_sample = []

total_run= 29
count = 1
for train_idx, val_idx in loo.split(X):
    # Extract the training and validation data
    X_train, y_train, Vin_train = X[train_idx], y[train_idx], Vin[train_idx]
    X_val, y_val, Vin_val = X[val_idx], y[val_idx], Vin[val_idx]

    # kfolf cross validation 
    best_params = hpo(cfg, task=task, X_train=X_train, y_train=y_train, Vin_train=Vin_train, n_distribution=n_distribution, fit_model=fit_model)
    
    # train and test using the best hyperparameters found with the kfold
    results = train_test_evaluation(cfg, task, X_train, X_val, y_train, y_val, Vin_train, Vin_val, n_distribution, fit_model, best_params, loo_count=count)
    

    # Inverse transform targets to extract metrics
    Vref_pred_te = np.matmul(np.array(results["test"]['Vref_pred']), fit_model.Pref.T)
    Vref_true = results["test"]['Vref_true']
    
    
    # save metrics in the a list (one for each left out observation)
    Q2_value = r2_metric(np.array(Vref_true), Vref_pred_te)
    Q2_loo.append(Q2_value)

    MAE_value = np.mean(np.abs(Vref_true-Vref_pred_te))
    MAE_loo.append(MAE_value)

    RMSE_value = np.sqrt(np.mean(np.square(Vref_true-Vref_pred_te)))
    RMSE_loo.append(RMSE_value)

    NE_value = np.nan_to_num(np.linalg.norm(Vref_true - Vref_pred_te) / np.linalg.norm(Vref_true), posinf=0, neginf=0, nan=0)
    NE_loo.append(NE_value)

    Vref_true_loo.append(np.array(Vref_true).flatten())
    Vref_pred_loo.append(np.array(Vref_pred_te).flatten())
    Vpred_final.append(np.array(results["test"]['Vref_pred']).flatten())
    Vin_reservoir_final.append(np.array(results["test"]['Vin_reservoir']).flatten())
    

    print(f'run {count}/{total_run}')
    print(f'Loss Train: {results["train"]["losses"]}')
    print(f'Loss Test: {results["test"]["losses"]}')
    print(f'Q²:{Q2_value}')
    print(f'MAE:{MAE_value}')
    print(f'RMSE {RMSE_value}')
    print(f'NE:{NE_value}')
    
    list_SV_loss_per_sample.append(results["test"]["losses"][1])
    count += 1
    



# extra 2 upper bound for mechanistic model

In [ ]:
df_input_pFBA = pd.DataFrame(np.maximum(0, Vin_reservoir_final), columns=['R_EX_etoh_e', 'R_EX_ac_e'])
df_input_pFBA